In [7]:
import numpy as np
from pathlib import Path
from skimage import io
from skimage import color
from skimage import measure
from skimage import morphology
from skimage import transform
from skimage import util
import re
import matplotlib.pyplot as plt
%matplotlib qt

In [ ]:
from collections import Counter
print(Counter(image_result.ravel()))

# Aktualne

### Pomocnicze

In [13]:
def detect_lines_of_text(img):    
    # obliczamy średnią jasność wierszy w obrazie
    sum_of_rows = np.sum(img, axis=1) 
    mean_row_value = np.mean(sum_of_rows)
    
    # wiersze poniżej średniej jasności zerujemy (usuwa np. ogonki liter nachodzących na kolejne wiersze)
    # chodzi o to żeby wyciągnąć na pewno te wiersze obrazu, w których jest tekst
    for i, row in enumerate(img):
#         print(np.sum(row))
        if np.sum(row) < mean_row_value: # lub mean_row_value, ale to jest takie niefajne, eh
            row = np.zeros(row.shape)
            img[i] = row
        
    # zaznaczamy obszary, które są powyżej średniej (całe wiersze)
    for i, row in enumerate(img):
#         print(np.mean(row))
        if np.sum(row) > 0.0:
            img[i:i+1, :] = 1 
            
    # łączenie lekko rozdzielonych linijek
#     img = morphology.dilation(img, morphology.disk(5)) # TODO WAŻNE - zoptymalizować!
                
    return img
            

def detect_words_in_line(image_result, image_binary, coords_of_line, row_intensity=255):
    # wycinamy kawałek obrazu będącego linią tekstu i obracamy go (.T)
    line_img = get_slice_of_image_with_specific_coords(image=image_binary, coords=coords_of_line).T
    line_img = morphology.dilation(line_img, morphology.disk(13))
    
    # szukamy miejsc, w których jasność jest większa od 0.0 i te miejsca zaznaczamy w wycinku obrazu
    # (obraz jest obrócony, więc tekst idzie z góry na dół)
    sum_of_rows = np.sum(line_img, axis=1) 
    mean_row_value = np.mean(sum_of_rows)
    for i, row in enumerate(line_img):
        if np.mean(row) > 0.0:
#             line_img[i:i+1, :] = (255, 0, 0)
            line_img[i:i+1, :] = row_intensity
    # znowu obracamy, tekst biegnie od lewej do prawej
    line_img = line_img.T
    
    # wykrywamy regiony, czyli pojedynczy region to powinien być jeden wyraz
    label_line_img = measure.label(line_img)
    regions = measure.regionprops(label_line_img)
#     print("liczba słów: ", len(regions))
    
    # tutaj szukamy regionu, który jest najdalej na lewo - czyli indeksu
    max_width_coord = max(regions[0].coords[:, 1])
    max_region_index = 0
    for i, region in enumerate(regions[1:]):        
        temp = max(region.coords[:, 1])
        if temp > max_width_coord:
            max_width_coord = temp
            max_region_index = i + 1
    
    # czyli mamy wszystkie współrzędne regionu z indeksem
    last_word_coords = regions[max_region_index].coords
    # ale aktualnie do tego regionu odnosimy się względem naszego wycinka obrazu - jednego wiersza
    # a chcemy go zaznaczyć na całym obrazie, więc do współrzędnych dodajemy współrzędne naszego wycinka,
    # (te współrzędne wycinka odnoszą się do całego obrazu)
    last_word_coords[:,0] += coords_of_line[0][0]
    
    # zamieniamy ten wycinek obrazu w całym obrazie
    first_point = coords_of_line[0]
    last_point = coords_of_line[-1]
    image_result[first_point[0]:last_point[0]+1, first_point[1]:last_point[1]+1] = line_img
                
    return last_word_coords


def get_slice_of_image_with_specific_coords(image, coords):
    height = coords[:, 0]
    width = coords[:, 1]
    slice_image = image[(height, width)].reshape((-1, image.shape[1]))
    
    return slice_image

In [26]:
p = Path("D:\Dokumenty\Projekty\PIRO\Projekt2\data\partial_results\canny_adam_2")
images_paths = source_path_canny.glob("(*.png)|(*.jpg)")
print(list(images_paths))

NotImplementedError: Non-relative patterns are unsupported

### Główny program

In [14]:
source_path_org = Path('../data/ocr1')
source_path_canny = Path('../data/partial_results/canny_adam_2')

save_path = Path('../data/partial_results/wyciete_indeksy_2')
save_path.mkdir(parents=True, exist_ok=True)

save_path_k_wyrazy = Path('../data/partial_results/k_wyrazy_2')
save_path_k_wyrazy.mkdir(parents=True, exist_ok=True)

images_paths = source_path_canny.glob("*.png")
all_k_wyrazy = []
for image_path in images_paths:
#     image_path = Path("../data/partial_results/canny_adam/img_1.png")
    number_of_image = re.search('[0-9]+', image_path.stem)[0]
    print(image_path)
    img_canny = io.imread(image_path)
    img_canny = color.rgb2gray(img_canny)
#     img_canny = morphology.closing(img_canny, morphology.square(3))
#     img_canny = morphology.opening(img_canny, morphology.disk(1))
  
    img_detected_rows = detect_lines_of_text(img_canny.copy()) 
    
#     io.imshow(img_detected_rows)
#     break
    
    # region = linia tekstu
    label_image = measure.label(img_detected_rows)
    regions = measure.regionprops(label_image)
    print("regions: ", len(regions))
    
#     width = img_canny.shape[1]
#     regions = [reg for reg in regions if reg.area > width*5] # wiersze powyżej 7 pikseli wysokości
#     print("regions po usunięciu cienkich wierszy: ", len(regions))
    
    
    image_result = np.zeros(img_canny.shape, dtype=np.uint8)
    last_words = []
    for i, region in enumerate(regions, 1):
        last_word_coords = detect_words_in_line(image_result=image_result, 
                                                           image_binary=img_canny, 
                                                           coords_of_line=region.coords, 
                                                           row_intensity=((i*1)%256))
        last_words.append(last_word_coords)
    
    # zapisywanie k-wyrazy        
    io.imsave(arr=image_result, fname=save_path_k_wyrazy / '{}-wyrazy.png'.format(number_of_image))
    all_k_wyrazy.append(image_result)
    
#     io.imshow(image_result)
#     break

    # Wczytanie oryginalnego obrazu i utworzenie katalogu dla wycinka indeksu.
    img_org = io.imread(source_path_org / (image_path.stem + ".jpg"))
    number_of_image = re.search('[0-9]+', image_path.stem)[0]
    last_word_directory = save_path / number_of_image
    last_word_directory.mkdir(parents=True, exist_ok=True)
    
    # Wycięcie indeksu (last_word) z oryginalnego obrazu i zapisanie go w katalogu dla danego obrazu.
    for i, last_word_coords in enumerate(last_words):
        first_point = last_word_coords[0]
        last_point = last_word_coords[-1]
        last_word_img = img_org[first_point[0]:last_point[0]+1, first_point[1]:last_point[1]+1] 
        io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
    



#     io.imshow(img_org)
#     break

..\data\partial_results\canny_adam_2\img_1.png
regions:  13


<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\1\0.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\1\1.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\1\2.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))


..\data\partial_results\canny_adam_2\img_10.png
regions:  14


<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\10\0.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\10\1.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\10\2.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\10\3.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\10\4.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.p

..\data\partial_results\canny_adam_2\img_11.png
regions:  10


...

<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\5\0.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\5\1.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\5\2.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\5\3.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\5\4.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.f

..\data\partial_results\canny_adam_2\img_6.png
regions:  15
..\data\partial_results\canny_adam_2\img_7.png
regions:  19


<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\7\0.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\7\1.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\7\2.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\7\3.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\7\4.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.f

..\data\partial_results\canny_adam_2\img_8.png
regions:  17
..\data\partial_results\canny_adam_2\img_9.png
regions:  11


<ipython-input-14-d3feef5aee87>:46: UserWarning: ..\data\partial_results\k_wyrazy_2\9-wyrazy.png is a low contrast image
  io.imsave(arr=image_result, fname=save_path_k_wyrazy / '{}-wyrazy.png'.format(number_of_image))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\9\0.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\9\1.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\9\2.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_directory / '{}.png'.format(i))
<ipython-input-14-d3feef5aee87>:63: UserWarning: ..\data\partial_results\wyciete_indeksy_2\9\3.png is a low contrast image
  io.imsave(arr=last_word_img, fname=last_word_direc

In [16]:
# kliknięcie ikony HOME (z domkiem) zmienia kolejne obrazy
source_path = Path('../data/partial_results/k_wyrazy_2')
image_paths = source_path.glob("*.png")

all_k_wyrazy = []
for image_path in image_paths:
    all_k_wyrazy.append(io.imread(image_path))

cnt = 0
def callback_home_button(event):
    ''' this function gets called if we hit the home button'''
    global cnt, image_paths
    cnt+=1
    image_paths = source_path.glob("*.png")
    print(list(image_paths)[cnt])
#     print(source_path / "{}".format(image_paths[cnt].name))
#     ss.set_data(all_k_wyrazy[cnt])
    plt.imshow(all_k_wyrazy[cnt], interpolation='nearest')
    plt.draw()
    
fig = plt.figure()

toolbar_elements = fig.canvas.toolbar.children()
right_button = toolbar_elements[4]

right_button.clicked.connect(callback_home_button)
ss = plt.imshow(all_k_wyrazy[cnt], interpolation='nearest')
print("CHYBA: ", source_path / "{}-wyrazy".format(cnt))

CHYBA:  ..\data\partial_results\k_wyrazy_2\0-wyrazy
..\data\partial_results\k_wyrazy_2\10-wyrazy.png
..\data\partial_results\k_wyrazy_2\11-wyrazy.png
..\data\partial_results\k_wyrazy_2\12-wyrazy.png
..\data\partial_results\k_wyrazy_2\13-wyrazy.png
..\data\partial_results\k_wyrazy_2\14-wyrazy.png
..\data\partial_results\k_wyrazy_2\15-wyrazy.png
..\data\partial_results\k_wyrazy_2\16-wyrazy.png
..\data\partial_results\k_wyrazy_2\17-wyrazy.png
..\data\partial_results\k_wyrazy_2\18-wyrazy.png
..\data\partial_results\k_wyrazy_2\19-wyrazy.png
..\data\partial_results\k_wyrazy_2\2-wyrazy.png
..\data\partial_results\k_wyrazy_2\20-wyrazy.png
..\data\partial_results\k_wyrazy_2\21-wyrazy.png
..\data\partial_results\k_wyrazy_2\22-wyrazy.png
..\data\partial_results\k_wyrazy_2\23-wyrazy.png
..\data\partial_results\k_wyrazy_2\24-wyrazy.png
..\data\partial_results\k_wyrazy_2\25-wyrazy.png
..\data\partial_results\k_wyrazy_2\26-wyrazy.png
..\data\partial_results\k_wyrazy_2\27-wyrazy.png
..\data\partial_re

Jakie problemy?
- 10, 14 - wiersze zbyt blisko sobie i łączą się w jeden oraz wykrywa jakieś szumy po bokach
- 12, 14 - łączenie wyrazów zbyt blisko siebie
- 17 - w ostatniej linijce rozdzielone słowo
- 18 - gdy napisany są ukośnie to bardzo źle wychodzi, bo wiersze się łączą
- 19 - indeksy rozdzielone, bo duża przerwa między cyframi

In [19]:
i = 32
type(i) == int
    

True

int